In [1]:
import os
from os.path import join
from tqdm.notebook import tqdm
import multiprocessing

import numpy as np
import pandas as pd

# Path
ROOT_DIR_PATH = '../D_WEATHER/input'
TRAIN_DIR_PATH = join(ROOT_DIR_PATH, 'train')
TEST_DIR_PATH = join(ROOT_DIR_PATH, 'test')
TRAIN_FEATHER_PATH = join(ROOT_DIR_PATH, 'train.ftr')
TEST_FEATHER_PATH = join(ROOT_DIR_PATH, 'test.ftr')

# Column(feature) names
INPUT_COL = [f"temp{i}" for i in range(1, 10)] + ['type', 'long_GMI', 'lat_GMI', 'long_DPR', 'lat_DPR']
TARGET_COL = ['precipitation']
TEST_COL = INPUT_COL
TRAIN_COL = INPUT_COL + TARGET_COL

# Name column (optional)
IS_PAD_NAME_COL = False
NAME_COL = ['orbit', 'subset', 'pixel']
PIXEL_COL = np.arange(1, 1601)[:, None]


def pad_name_cols(nd, file_name):
    orbit, subset = file_name.split('_')[1:]
    subset = subset[:2]
    nd = np.pad(nd, ((0, 0), (0, 1)), constant_values=int(orbit))
    nd = np.pad(nd, ((0, 0), (0, 1)), constant_values=int(subset))
    return np.c_[nd, PIXEL_COL]

def generate_ndarray_from_file_name(file_name, dir_path):
    file_path = join(dir_path, file_name)
    nd = np.load(file_path).astype(np.float32)  # 40 x 40 x ?
    dim = nd.shape[-1]
    nd = nd.reshape(-1, dim)                    # 1600    x ?
    if IS_PAD_NAME_COL:
        nd = pad_name_col(nd, file_name)
    return nd
    
def generate_ndarray_from_dir_path(dir_path):
    pool = multiprocessing.Pool()
    nds = pool.starmap(generate_ndarray_from_file_name, [(file_name, dir_path) for file_name in tqdm(os.listdir(dir_path))])
    return np.concatenate(nds)


def generate_dataframe_from_dir_path(dir_path):
    nd = generate_ndarray_from_dir_path(dir_path)
    dim = nd.shape[-1]
    df =  pd.DataFrame(nd,
                       columns=TRAIN_COL if dim == len(TRAIN_COL) else TEST_COL,
                       dtype=np.float32
                      )
    if IS_PAD_NAME_COL:
        df[['orbit', 'subset', 'pixel']] = df[['orbit', 'subset', 'pixel']].astype(np.int32)
        df.sort_values(by=['orbit', 'subset', 'pixel'], ignore_index=True, inplace=True)
    return df


def main():
    global TRAIN_COL, TEST_COL
    TRAIN_COL, TEST_COL = (TRAIN_COL, TEST_COL) if not IS_PAD_NAME_COL else (TRAIN_COL + NAME_COL, TEST_COL + NAME_COL)
    for dir_path in (TRAIN_DIR_PATH, TEST_DIR_PATH):
        df = generate_dataframe_from_dir_path(dir_path)
        if len(df.columns) == len(TRAIN_COL):
            train_df = df
        else:
            test_df = df
    return train_df, test_df

def to_feather(train_df, test_df):
    train_df.to_feather(TRAIN_FEATHER_PATH)
    test_df.to_feather(TEST_FEATHER_PATH)

def read_feather():
    train_df = pd.read_feather(TRAIN_FEATHER_PATH)
    test_df = pd.read_feather(TEST_FEATHER_PATH)
    return train_df, test_df

!pip install memory-profiler

!pip install line-profiler

In [5]:
%load_ext memory_profiler
%load_ext line_profiler

In [6]:
%%time
%memit train_df, test_df = main()
# 14s / 15GB


peak memory: 14509.57 MiB, increment: 14400.88 MiB
CPU times: user 2.55 s, sys: 8.49 s, total: 11 s
Wall time: 36.5 s


In [7]:
%%time
%memit to_feather(train_df, test_df)

peak memory: 21594.48 MiB, increment: 7220.54 MiB
CPU times: user 1min 32s, sys: 13.3 s, total: 1min 45s
Wall time: 57.8 s


In [8]:
%%time
%memit train_df, test_df = read_feather()

peak memory: 28330.79 MiB, increment: 13950.77 MiB
CPU times: user 3.48 s, sys: 6.71 s, total: 10.2 s
Wall time: 28.6 s


In [9]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122152000 entries, 0 to 122151999
Data columns (total 15 columns):
temp1            float32
temp2            float32
temp3            float32
temp4            float32
temp5            float32
temp6            float32
temp7            float32
temp8            float32
temp9            float32
type             float32
long_GMI         float32
lat_GMI          float32
long_DPR         float32
lat_DPR          float32
precipitation    float32
dtypes: float32(15)
memory usage: 6.8 GB


,temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,type,long_GMI,lat_GMI,long_DPR,lat_DPR,precipitation
0,276.345306,269.061676,275.768982,267.590973,278.918488,276.057617,267.718536,281.058044,276.297760,105.0,111.984665,27.743324,112.112061,27.672855,0.0
1,274.954651,266.842010,275.510803,266.518219,279.108368,276.858185,268.559814,282.791229,279.515442,105.0,112.028824,27.709145,112.112061,27.672855,0.0
2,273.050293,264.990906,274.355103,264.183319,278.565918,276.398529,267.664001,282.788483,279.462555,105.0,112.073425,27.675425,112.112061,27.672855,0.0
3,273.705475,264.647308,274.066589,263.972961,278.802643,275.811066,267.545715,282.927765,280.419617,102.0,112.118446,27.642170,112.091187,27.632555,0.0
4,273.936523,265.798096,275.803589,268.571960,279.572601,277.099731,269.820526,282.501099,280.374359,102.0,112.163895,27.609385,112.139694,27.607765,0.0


In [11]:
train_df.describe()

,temp1,temp2,temp3,temp4,temp5,temp6,temp7,temp8,temp9,type,long_GMI,lat_GMI,long_DPR,lat_DPR,precipitation
count,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08,1.221520e+08
mean,1.973029e+02,1.399296e+02,2.171046e+02,1.696791e+02,2.395907e+02,2.333365e+02,1.921457e+02,2.643873e+02,2.458601e+02,3.196320e+01,1.395868e+02,3.091759e+01,1.395868e+02,3.091750e+01,-4.436900e+01
std,4.340960e+01,7.234476e+01,3.394657e+01,5.850119e+01,2.687159e+01,2.432388e+01,4.663368e+01,1.897327e+01,2.962355e+01,5.410535e+01,1.981270e+01,1.427054e+01,1.981248e+01,1.427062e+01,6.657127e+02
min,1.016668e+02,4.582756e+01,1.666451e+02,9.073691e+01,1.686818e+02,1.279539e+02,1.196357e+02,6.470033e+01,6.310448e+01,0.000000e+00,1.036093e+02,4.078568e+00,1.035896e+02,2.442523e+00,-9.999900e+03
25%,1.695079e+02,9.282653e+01,1.926689e+02,1.271468e+02,2.215650e+02,2.156677e+02,1.573114e+02,2.533665e+02,2.277993e+02,0.000000e+00,1.224348e+02,1.860782e+01,1.224392e+02,1.860756e+01,0.000000e+00
50%,1.739428e+02,9.710215e+01,2.059318e+02,1.466576e+02,2.432162e+02,2.264670e+02,1.748229e+02,2.708615e+02,2.556745e+02,0.000000e+00,1.398330e+02,3.128326e+01,1.398316e+02,3.128290e+01,0.000000e+00
75%,2.466735e+02,2.267323e+02,2.472664e+02,2.325092e+02,2.557330e+02,2.490251e+02,2.333685e+02,2.782477e+02,2.675714e+02,1.030000e+02,1.567187e+02,4.345076e+01,1.567155e+02,4.345127e+01,0.000000e+00
max,1.576792e+03,1.531369e+03,1.173939e+03,1.131579e+03,3.422480e+02,7.420804e+02,4.525078e+02,3.084020e+02,3.696122e+02,3.220000e+02,1.762840e+02,5.584636e+01,1.762885e+02,5.582747e+01,2.999978e+02


In [12]:
train_df[train_df['precipitation']<0].shape

(543756, 15)